In [1]:
import json

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

from myrep import ControlVector, ControlModel, DatasetEntry, Experiment

/Users/bertram/.pyenv/versions/3.10.0/envs/repa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"

tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model = model.to("cuda:0" if torch.cuda.is_available() else "mps:0" if torch.backends.mps.is_available() else "cpu")
#model = ControlModel(model, list(range(-10, -15, -1)))

#user_tag, asst_tag = "[INST]", "[/INST]"

In [ ]:
model.model.layers

In [ ]:
model.gpt_neox.layers

In [ ]:
with open("data/all_truncated_outputs.json") as f:
    suffixes = json.load(f)

# you don't need 3 here, you can have as few as one each.
# make sure they are closely matched, however—they should be direct opposites if possible.
# bad: "high on acid" / "sober" — "sober" implies alcohol, so you don't get a clean vector
# good: "high on acid" / "sober, not on acid" — the negative prompt is more directly opposite
positive_personas = ["loving"]
negative_personas = ["hateful"]

def template(persona: str, suffix: str) -> str:
    return f"{user_tag} Act as if you're extremely {persona}. {asst_tag} {suffix}"

dataset = []
for suffix in suffixes:
    tokens = tokenizer.tokenize(suffix)
    for i in range(1, len(tokens)):
        truncated = tokenizer.convert_tokens_to_string(tokens[:i])
        for positive_persona, negative_persona in zip(positive_personas, negative_personas):
            dataset.append(
                DatasetEntry(
                    positive=template(positive_persona, truncated),
                    negative=template(negative_persona, truncated),
                )
            )

# print some example entries
for i in range(3):
    print(f"dataset[{i}].positive:", dataset[i].positive)
    print(f"dataset[{i}].negative:", dataset[i].negative)

In [ ]:
model.reset() # make sure you always reset the model before training a new vector
control_vector = ControlVector.train(
    model,
    tokenizer,
    dataset,
)

In [ ]:
control_vector

In [ ]:
# the question to ask the modified model
# don't forget the space after {user_tag} and before {asst_tag}!
#input = f"{user_tag} How has your day been? {asst_tag}"
input = f"How has your day been?"

# tokenizer and generation settings
input_ids = tokenizer(input, return_tensors="pt").to(model.device)
settings = {
    "pad_token_id": tokenizer.eos_token_id, # silence warning
    "do_sample": False, # temperature=0
    "max_new_tokens": 128,
    "repetition_penalty": 1.1, # reduce control jank
}

print("==baseline")
model.reset()
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n++control")
# add the control vector with a certain strength (try increasing or decreasing this!)
model.set_control(control_vector, 5)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n--control")
# subtract the control vector, giving the opposite result (e.g. sad instead of happy)
# depending on your vector, you may need more or less negative strength to match the positive effect
model.set_control(control_vector, -5)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))
model.reset()